In [2]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from datetime import datetime,timedelta
import plotly.express as px
from numba import njit

In [3]:
import vectorbtpro as vbt
vbt.settings.set_theme("dark")

In [12]:
data = vbt.TVData.fetch('NASDAQ:GOOG', timeframe = '1 day')    
#initialize dataFrame
df = pd.DataFrame(data.get())
# df = df.round(2)
df.index = pd.to_datetime(df.index.date)

# signal development


# 1 EXPRESSION

In [4]:
MaskGenerator = vbt.IF.from_expr("""
upperband, middleband, lowerband = @res_talib_bbands
real = @res_talib_ema
bandwidth = (upperband - lowerband) / middleband
cond1 = low < lowerband
cond2 = bandwidth > @p_cond2_th
cond3 = high > upperband
cond4 = bandwidth < @p_cond4_th
@out_mask:(cond1 & cond2) | (cond3 & cond4)
""")


In [5]:
# ind = pd.DataFrame()
# ind.ta.indicators()
# vbt.IF.list_indicators()

# check pandas ta
# vbt.IF.list_pandas_ta_indicators()[:]

In [6]:
# vbt.phelp(MaskGenerator.run)

# vbt.phelp(vbt.IF.from_pandas_ta('EMA').run)

# vbt.phelp(vbt.IF.from_talib('RSI').run)
vbt.phelp(vbt.IF.from_talib('BBANDS').run)

BBANDS.run(
    close,
    timeperiod=Default(value=5),
    nbdevup=Default(value=2),
    nbdevdn=Default(value=2),
    matype=Default(value=0),
    timeframe=Default(value=None),
    short_name='bbands',
    hide_params=None,
    hide_default=True,
    **kwargs
):
    Run `BBANDS` indicator.
    
    * Inputs: `close`
    * Parameters: `timeperiod`, `nbdevup`, `nbdevdn`, `matype`, `timeframe`
    * Outputs: `upperband`, `middleband`, `lowerband`
    
    Pass a list of parameter names as `hide_params` to hide their column levels, or True to hide all.
    Set `hide_default` to False to show the column levels of the parameters with a default value.
    
    Other keyword arguments are passed to `BBANDS.run_pipeline`.


# 2 Function Numba

In [7]:
# def generate_mask_1d_nb(  
#     high, low,  
#     uband, mband, lband,  
#     cond2_th, cond4_th  
# ):
#     out = np.full(high.shape, False)  
    
#     for i in range(high.shape[0]):  
        
#         bandwidth = (uband[i] - lband[i]) / mband[i]
#         cond1 = low[i] < lband[i]
#         cond2 = bandwidth > cond2_th
#         cond3 = high[i] > uband[i]
#         cond4 = bandwidth < cond4_th
#         signal = (cond1 and cond2) or (cond3 and cond4)  
#         out[i] = signal  
        
#     return out

# mask = generate_mask_1d_nb(
#     data.get("High")["BTCUSDT"].values,  
#     data.get("Low")["BTCUSDT"].values,
#     bb.upperband["BTCUSDT"].values,
#     bb.middleband["BTCUSDT"].values,
#     bb.lowerband["BTCUSDT"].values,
#     0.30,
#     0.15
# )
# symbol_wrapper = data.get_symbol_wrapper()
# mask = symbol_wrapper["BTCUSDT"].wrap(mask)  
# mask.sum()

In [8]:
# @njit
# def generate_mask_nb(  
#     high, low,
#     uband, mband, lband,
#     cond2_th, cond4_th
# ):
#     out = np.empty(high.shape, dtype=np.bool_)  
    
#     for col in range(high.shape[1]):  
#         out[:, col] = generate_mask_1d_nb(  
#             high[:, col], low[:, col],
#             uband[:, col], mband[:, col], lband[:, col],
#             cond2_th, cond4_th
#         )
        
#     return out

# mask = generate_mask_nb(
#     vbt.to_2d_array(data.get("High")),  
#     vbt.to_2d_array(data.get("Low")),
#     vbt.to_2d_array(bb.upperband),
#     vbt.to_2d_array(bb.middleband),
#     vbt.to_2d_array(bb.lowerband),
#     0.30,
#     0.15
# )

In [9]:
# MaskGenerator = vbt.IF(  
#     input_names=["high", "low", "uband", "mband", "lband"],
#     param_names=["cond2_th", "cond4_th"],
#     output_names=["mask"]
# ).with_apply_func(generate_mask_1d_nb, takes_1d=True)  
# mask_generator = MaskGenerator.run(  
#     data.get("High"),
#     data.get("Low"),
#     bb.upperband,
#     bb.middleband,
#     bb.lowerband,
#     [0.3, 0.4],
#     [0.1, 0.2],
#     param_product=True  
# )
# mask_generator.mask.sum()

In [ ]:
# # example

# RSI = vbt.IF.from_talib('RSI')

# @njit
# def produce_signal(rsi, entry, exit):
#     trend = np.where(rsi > exit, -1, 0)
#     trend = np.where(rsi < entry, 1, trend)
#     return trend

# def custom_indicator(close, rsi_window=14, entry=30, exit=70):
#     rsi = RSI.run(close, rsi_window).real.to_numpy()
#     return produce_signal(rsi, entry, exit)

# ind = vbt.IF(
#     class_name="Combination",
#     short_name="comb",
#     input_names=["close"],
#     param_names=["rsi_window", "entry", "exit"],
#     output_names=["value"],
# ).with_apply_func(
#     custom_indicator,
#     rsi_window=14,
#     entry=30,
#     exit=70)

# res = ind.run(
#     df["Close"],
#     rsi_window=np.arange(2, 40, step=2, dtype=int),
#     entry=np.arange(10, 40, step=2, dtype=int),
#     exit=np.arange(60, 90, step=2, dtype=int),
#     param_product=True,
# )

# print(res.value)

# entries = res.value == 1
# exits = res.value == -1

# pf = vbt.Portfolio.from_signals(df["Close"],
#     entries,
#     exits,
#     freq='1h',
#     sl_stop=[0.02],
# )

# print(pf.stats())
# pf.plot().show()

# implement

# 1 

In [10]:
MaskGenerator = vbt.IF.from_expr("""
upperband, middleband, lowerband = @res_talib_bbands
bandwidth = (upperband - lowerband) / middleband
cond1 = low < lowerband
cond2 = bandwidth > @p_cond2_th
cond3 = high > upperband
cond4 = bandwidth < @p_cond4_th
@out_mask:(cond1 & cond2) | (cond3 & cond4)
""")

In [13]:
# vbt.phelp(vbt.IF.from_talib('RSI').run)
# vbt.phelp(vbt.IF.from_talib('BBANDS').run)
rsigen = vbt.IF.from_expr("""
real = @res_talib_rsi
cond1 = real < @p_oversold
cond2 = real > @p_overbought
@out_entries: real
""")

rsi_real = vbt.RSI.run(close=df['Close'],window=14) 
print(rsi_real.rsi)
# vbt.phelp(rsigen.run)   
ob = pd.DataFrame(np.full(shape=2276, fill_value=70).T )
os = pd.DataFrame(np.full(shape=2276, fill_value=30).T )
ob
ob2 = np.array([30 for i in range(2276)])
print(ob2)
print(df['Close'])
df['ob'] = ob2
# df['ob']
# mask = rsigen.run(close=df['Close'], overbought=ob, oversold=os)
mask = rsigen.run(close=df['Close'], overbought=70, oversold=30)
# print(mask.rsi)
# cannot be 2 output

2014-03-27          NaN
2014-03-28          NaN
2014-03-31          NaN
2014-04-01          NaN
2014-04-02          NaN
                ...    
2023-04-04    61.885847
2023-04-05    61.376020
2023-04-06    67.977230
2023-04-10    62.315074
2023-04-11    60.023327
Name: (14, Close), Length: 2276, dtype: float64
[30 30 30 ... 30 30 30]
2014-03-27     27.847803
2014-03-28     27.924097
2014-03-31     27.773504
2014-04-01     28.281632
2014-04-02     28.273653
                 ...    
2023-04-04    105.120000
2023-04-05    104.950000
2023-04-06    108.900000
2023-04-10    106.950000
2023-04-11    106.120000
Name: Close, Length: 2276, dtype: float64


In [14]:
# vbt.phelp(vbt.IF.from_talib('RSI').run)
# vbt.phelp(vbt.IF.from_talib('EMA').run)
tmp = vbt.IF.from_expr("""
rsi = @res_talib_rsi
ema = @res_talib_ema
cond1 = rsi >= 30
cond2 = ema >= 50
cond3 = rsi <= 70
cond4 = ema <= 200
@out_signal: cond1 & cond2 | cond3 & cond4
""")
ema = vbt.IF.from_talib('EMA').run(close=df['Close'], timeperiod = 50)
# print(ema.real)
tmp_strategy = tmp.run(close=df['Close'], rsi_timeperiod = 30, timeframe = 100)
vbt.phelp(tmp.run)

entries = tmp_strategy.signal
# tmp_strategy.signal
pf = vbt.Portfolio.from_signals(open = df["Open"], close = df["Close"], high = df["High"], low = df["Low"], 
                                    entries = entries, sl_stop = 0.1, tp_stop = 0.15, tsl_stop = 0.1,
                                    size = 1, min_size = 1, price = 'NextOpen', stop_entry_price = 'FillPrice')
# pf.stats()
# ema golden cannot approve

Indicator.run(
    close,
    rsi_timeperiod=Default(value=14),
    rsi_timeframe=Default(value=None),
    ema_timeperiod=Default(value=30),
    ema_timeframe=Default(value=None),
    short_name='custom',
    hide_params=None,
    hide_default=True,
    **kwargs
):
    Run `Indicator` indicator.
    
    * Inputs: `close`
    * Parameters: `rsi_timeperiod`, `rsi_timeframe`, `ema_timeperiod`, `ema_timeframe`
    * Outputs: `signal`
    
    Pass a list of parameter names as `hide_params` to hide their column levels, or True to hide all.
    Set `hide_default` to False to show the column levels of the parameters with a default value.
    
    Other keyword arguments are passed to `Indicator.run_pipeline`.


# both entries and exits

In [ ]:
# @njit
# def entry_place_func_nb(c, low, close, th):
#     if c.from_i == 0:  
#         c.out[0] = True
#         return 0
#     exit_i = c.from_i - c.wait  
#     exit_price = close[exit_i, c.col]  
#     hit_price = exit_price * (1 - th)
#     for out_i in range(len(c.out)):
#         i = c.from_i + out_i
#         if low[i, c.col] <= hit_price:  
#             return out_i
#     return -1

# @njit
# def exit_place_func_nb(c, high, close, th):  
#     entry_i = c.from_i - c.wait
#     entry_price = close[entry_i, c.col]
#     hit_price = entry_price * (1 + th)
#     for out_i in range(len(c.out)):
#         i = c.from_i + out_i
#         if high[i, c.col] >= hit_price:
#             return out_i
#     return -1

# entries, exits = vbt.pd_acc.signals.generate_both(  
#     symbol_wrapper.shape,
#     entry_place_func_nb=entry_place_func_nb,
#     entry_args=(vbt.Rep("low"), vbt.Rep("close"), 0.1),  
#     exit_place_func_nb=exit_place_func_nb,
#     exit_args=(vbt.Rep("high"), vbt.Rep("close"), 0.2),
#     wrapper=symbol_wrapper,
#     broadcast_named_args=dict(  
#         high=data.get("High"),
#         low=data.get("Low"),
#         close=data.get("Close")
#     ),
#     broadcast_kwargs=dict(post_func=np.asarray)  
# )

# fig = data.plot(
#     symbol="BTCUSDT", 
#     ohlc_trace_kwargs=dict(opacity=0.5), 
#     plot_volume=False
# )
# entries["BTCUSDT"].vbt.signals.plot_as_entries(
#     y=data.get("Close", "BTCUSDT"), fig=fig)
# exits["BTCUSDT"].vbt.signals.plot_as_exits(
#     y=data.get("Close", "BTCUSDT"), fig=fig)
# fig.show()  

# 2

In [31]:
# example

RSI = vbt.IF.from_talib('RSI')

@njit
def produce_signal(rsi, entry, exit):
    trend = np.where(rsi > exit, -1, 0)
    trend = np.where(rsi < entry, 1, trend)
    return trend

def custom_indicator(close, rsi_window=14, entry=30, exit=70):
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    return produce_signal(rsi, entry, exit)

ind = vbt.IF(
    class_name="Combination",
    short_name="comb",
    input_names=["close"],
    param_names=["rsi_window", "entry", "exit"],
    output_names=["value"],
).with_apply_func(
    custom_indicator,
    rsi_window=14,
    entry=30,
    exit=70)
# .with_apply_func => set to default
res = ind.run(
    df["Close"],
    rsi_window=14,
    entry=30,
    exit=70,
)

print(res.value)

entries = res.value == 1
exits = res.value == -1

pf = vbt.Portfolio.from_signals(df["Close"],
    entries,
    exits,
    freq='1h',
    sl_stop=[0.02],
)

print(pf.stats())
pf.plot().show()

2014-03-27    0
2014-03-28    0
2014-03-31    0
2014-04-01    0
2014-04-02    0
             ..
2023-04-04    0
2023-04-05    0
2023-04-06    0
2023-04-10    0
2023-04-11    0
Name: (14, 30, 70, Close), Length: 2276, dtype: int64
Start                         2014-03-27 00:00:00
End                           2023-04-11 00:00:00
Period                           94 days 20:00:00
Start Value                                 100.0
Min Value                               97.438178
Max Value                              198.682448
End Value                              198.212425
Total Return [%]                        98.212425
Benchmark Return [%]                   281.071354
Total Time Exposure [%]                 13.224956
Max Gross Exposure [%]                      100.0
Max Drawdown [%]                        15.689744
Max Drawdown Duration            28 days 12:00:00
Total Orders                                   24
Total Fees Paid                               0.0
Total Trades        

## 1. expression
- combine indicator
- ema golden cannot approve
- cannot 2 outputs
## 2. with_apply_function
- combine indicator
- better than expression
- njit
- numpy